# HackUPC - HP Supply Chain Optimization
### Team: Baseline 

The aim of this notebook is to to predict the inventory for the next 13 weeks based on the dataset provided by HP.

What will be covered:


1.   Install Libraries
2.   Import Libraries
3.   Import Dataset
4. Data Processing
5. Handling Missing Data
6. Feature Engineering
7. Data Preparation
8. Model Training
9. Inference




### Install Libraries

In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import Libraries

In [ ]:

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, LSTM
import tensorflow_addons as tfa
import os
import tensorflow as tf
from tensorflow.keras import layers
import datetime
%matplotlib inline

### Import Dataset

In [ ]:
data = pd.read_csv('/content/train.csv')
data

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0
2,202121-6909,2021-05-29,202121,6909,93,Arale,21274,13.3,Premium,3.0,137.0
3,202122-6909,2021-06-05,202122,6909,93,Arale,21274,13.3,Premium,0.0,274.0
4,202123-6909,2021-06-12,202123,6909,93,Arale,21274,13.3,Premium,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...
15449,202317-247737,2023-04-29,202317,247737,15,Arale,10637,13.3,Premium,0.0,62.0
15450,202317-247737,2023-04-29,202317,247737,24,Arale,10637,13.3,Premium,0.0,3.0
15451,202318-247737,2023-05-06,202318,247737,3,Arale,10637,13.3,Premium,22.0,0.0
15452,202318-247737,2023-05-06,202318,247737,15,Arale,10637,13.3,Premium,68.0,78.0


# Data Processing



Check and correct the column data types

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15454 entries, 0 to 15453
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               15454 non-null  object 
 1   date             15454 non-null  object 
 2   year_week        15454 non-null  int64  
 3   product_number   15454 non-null  int64  
 4   reporterhq_id    15454 non-null  int64  
 5   prod_category    15454 non-null  object 
 6   specs            15454 non-null  int64  
 7   display_size     15454 non-null  float64
 8   segment          15454 non-null  object 
 9   sales_units      14963 non-null  float64
 10  inventory_units  14961 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 1.3+ MB


In [ ]:
# Define the column type mapping
cate_columns = ['id', 'year_week', 'product_number', 'reporterhq_id',
       'prod_category', 'specs', 'display_size', 'segment']
data[cate_columns] = data[cate_columns].astype('str')

In [ ]:
# Cast date type
data['date'] = pd.to_datetime(data['date'])

In [ ]:
data.dtypes

id                         object
date               datetime64[ns]
year_week                  object
product_number             object
reporterhq_id              object
prod_category              object
specs                      object
display_size               object
segment                    object
sales_units               float64
inventory_units           float64
dtype: object

Because the dataset has alot of missing data of year week, we decided to filter some product data after analyzing.

In [ ]:
df = data.copy()

In [ ]:
dfs = []
config = {
    "236880":"202201",
    "148050":"202243",
    "12831":"202221",
    "48363":"202233",
    "157920":"202228",
    "190491":"202227",
    "46389":"202204",
    "116466":"202217",
    "234906":"202204"
}

for product_id in product_ids:
  if product_id in list(config.keys()):
    filtered_df = df[(df['product_number'] == product_id) & (df['year_week'] >= config[product_id]) & (df['year_week'] <= '202318')]
    dfs.append(filtered_df)
  dfs.append(df[df['product_number'] == product_id])

In [ ]:
# Concat the dataframes
df_col_merged = pd.concat(dfs, axis=0)
df_col_merged

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0
2,202121-6909,2021-05-29,202121,6909,93,Arale,21274,13.3,Premium,3.0,137.0
3,202122-6909,2021-06-05,202122,6909,93,Arale,21274,13.3,Premium,0.0,274.0
4,202123-6909,2021-06-12,202123,6909,93,Arale,21274,13.3,Premium,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...
15449,202317-247737,2023-04-29,202317,247737,15,Arale,10637,13.3,Premium,0.0,62.0
15450,202317-247737,2023-04-29,202317,247737,24,Arale,10637,13.3,Premium,0.0,3.0
15451,202318-247737,2023-05-06,202318,247737,3,Arale,10637,13.3,Premium,22.0,0.0
15452,202318-247737,2023-05-06,202318,247737,15,Arale,10637,13.3,Premium,68.0,78.0


In [ ]:
# Group by data based on some features below
df_col_mergedd = df_col_merged.groupby(['date', 'year_week', 'product_number', 'prod_category', 'display_size', 'segment']).agg({'sales_units': 'sum', 'inventory_units':'sum'}).reset_index()
df_col_mergedd

,date,year_week,product_number,prod_category,display_size,segment,sales_units,inventory_units
0,2019-04-13,201915,233919,Clover,15.6,Core,2.0,20.0
1,2019-04-20,201916,233919,Clover,15.6,Core,3.0,43.0
2,2019-04-27,201917,233919,Clover,15.6,Core,122.0,184.0
3,2019-05-04,201918,233919,Clover,15.6,Core,61.0,219.0
4,2019-05-11,201919,233919,Clover,15.6,Core,33.0,133.0
...,...,...,...,...,...,...,...,...
7704,2023-05-06,202318,92778,Goku,14.0,Core,36.0,92.0
7705,2023-05-06,202318,93765,Arale,14.0,Premium,47.0,31.0
7706,2023-05-06,202318,94752,Clover,14.0,Core,11.0,374.0
7707,2023-05-06,202318,9870,Doraemon,15.6,Gaming,10.0,111.0


### Handling Missing Data

In [ ]:
# Generate the year week from start year, week to end year, week
def generate_year_week_list(start_year, start_week, end_year=2023, end_week=18):
    start_date = datetime.datetime.strptime(f'{start_year}-W{start_week}-1', '%Y-W%W-%w')
    end_date = datetime.datetime.strptime(f'{end_year}-W{end_week}-1', '%Y-W%W-%w')
    current_date = start_date
    year_week_list = []
    
    while current_date <= end_date:
        year = current_date.isocalendar()[0]
        week = current_date.isocalendar()[1]
        year_week_list.append((year, f'{week:02}'))  # Format week number with leading zeros
        current_date += datetime.timedelta(weeks=1)

    df = pd.DataFrame(year_week_list, columns=['Year', 'Week'])
  
    return df

we fill the missing year week for each product.

In [ ]:
dfs = []
for product_id in df_col_mergedd.product_number.unique():
    product_data = df_col_mergedd[df_col_mergedd['product_number']==product_id]

    product_data = product_data.sort_values("year_week", ascending=True)
    year_week = product_data.head(1)['year_week'].values[0]

    start_year = year_week[:4]
    start_week = year_week[-2:]

    generated_data = generate_year_week_list(int(start_year),int(start_week))
    generated_data["year_week"] = generated_data["Year"].astype(str) + generated_data['Week'].astype(str)
    
    data = generated_data.merge(da, on='year_week', how='left')
    dfs.append(data)

In [ ]:
# Concat the dataframe
df_col = pd.concat(dfs, axis=0)
df_col

,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units
0,2019,16,201916,2019-04-20,233919,Clover,15.6,Core,3.0,43.0
1,2019,17,201917,2019-04-27,233919,Clover,15.6,Core,122.0,184.0
2,2019,18,201918,2019-05-04,233919,Clover,15.6,Core,61.0,219.0
3,2019,19,201919,2019-05-11,233919,Clover,15.6,Core,33.0,133.0
4,2019,20,201920,2019-05-18,233919,Clover,15.6,Core,142.0,310.0
...,...,...,...,...,...,...,...,...,...,...
27,2023,14,202314,2023-04-08,107583,Clover,15.6,Core,16.0,504.0
28,2023,15,202315,2023-04-15,107583,Clover,15.6,Core,77.0,245.0
29,2023,16,202316,2023-04-22,107583,Clover,15.6,Core,136.0,443.0
30,2023,17,202317,2023-04-29,107583,Clover,15.6,Core,115.0,297.0


Checking missing data

In [ ]:
df_col.isna().sum()

Year                 0
Week                 0
year_week            0
date               352
product_number     352
prod_category      352
display_size       352
segment            352
sales_units        352
inventory_units    352
dtype: int64

In [ ]:
# Fill the data
df_col.fillna(method='ffill', inplace = True)

In [ ]:
df_col.isna().sum()

Year               0
Week               0
year_week          0
date               0
product_number     0
prod_category      0
display_size       0
segment            0
sales_units        0
inventory_units    0
dtype: int64

After updating missing values, we have some date duplicate, we deal with it by adding 7 days for duplicate date

In [ ]:
df_col['date'] = pd.to_datetime(df_col['date'])

In [ ]:
def fix_date(data):
  sample = data.sort_values('year_week',ascending=True).reset_index(drop=True)
  for i in range(len(sample)-1):
    sample['date'][i+1] = sample['date'][i] + datetime.timedelta(weeks = 1)
  return sample

In [ ]:
# Fix date for all product dataset
dfs = []
for product_id in df_col.product_number.unique():
    product_data = df_col[df_col['product_number']==product_id]
    df = fix_date(product_data)
    dfs.append(df)

In [ ]:
# Concat DataFrame
df_col = pd.concat(dfs, axis=0)
df_col

,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units
0,2019,16,201916,2019-04-20,233919,Clover,15.6,Core,3.0,43.0
1,2019,17,201917,2019-04-27,233919,Clover,15.6,Core,122.0,184.0
2,2019,18,201918,2019-05-04,233919,Clover,15.6,Core,61.0,219.0
3,2019,19,201919,2019-05-11,233919,Clover,15.6,Core,33.0,133.0
4,2019,20,201920,2019-05-18,233919,Clover,15.6,Core,142.0,310.0
...,...,...,...,...,...,...,...,...,...,...
27,2023,14,202314,2023-04-08,107583,Clover,15.6,Core,16.0,504.0
28,2023,15,202315,2023-04-15,107583,Clover,15.6,Core,77.0,245.0
29,2023,16,202316,2023-04-22,107583,Clover,15.6,Core,136.0,443.0
30,2023,17,202317,2023-04-29,107583,Clover,15.6,Core,115.0,297.0


In [ ]:
# Save the data for visualing on PowerBI purposes
df_col.to_csv('final_data.csv',index=False)

In [ ]:
# Read The data
inventory_data = pd.read_csv('/content/final_data.csv', parse_dates=["date"], index_col=["date"]).sort_index()
inventory_data 

,Year,Week,year_week,product_number,prod_category,display_size,segment,sales_units,inventory_units
date,,,,,,,,,
2019-04-20,2019,16,201916,233919,Clover,15.6,Core,3.0,43.0
2019-04-27,2019,17,201917,233919,Clover,15.6,Core,122.0,184.0
2019-05-04,2019,18,201918,233919,Clover,15.6,Core,61.0,219.0
2019-05-11,2019,19,201919,233919,Clover,15.6,Core,33.0,133.0
2019-05-18,2019,20,201920,233919,Clover,15.6,Core,142.0,310.0
...,...,...,...,...,...,...,...,...,...
2023-05-06,2023,18,202318,82908,Luffy,14.0,Core,13.0,232.0
2023-05-06,2023,18,202318,196413,Clover,15.6,Core,90.0,148.0
2023-05-06,2023,18,202318,242802,Conan,13.3,Premium,62.0,57.0


In [ ]:
# Cast Datatype 
cate_columns = ['year_week', 'product_number', 'Year', 'Week',
       'prod_category', 'display_size', 'segment']
inventory_data[cate_columns] = inventory_data[cate_columns].astype('str')

### Feature Engineering
For each sample, we add the inventory of 4 previous weeks and their mean and standard deviation

In [ ]:
WINDOW_SIZE=4
list_data = []
for product_id in inventory_data.product_number.unique():
    data = df_col[df_col['product_number']==product_id]
    # Shift values for each step in WINDOW_SIZE
    for i in range(WINDOW_SIZE): 
        data[f"inventory_units+{i+1}"] = data["inventory_units"].shift(periods=i+1)
    # Add Mean
    data['Mean'] = data[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].mean(axis=1)
    # Add std
    data['std'] = data[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].std(axis=1)
    list_data.append(data)

In [ ]:
# Concat the data
clean_data = pd.concat(list_data, axis=0)
clean_data

,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units,inventory_units+1,inventory_units+2,inventory_units+3,inventory_units+4,Mean,std
0,2019,16,201916,2019-04-20,233919,Clover,15.6,Core,3.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,17,201917,2019-04-27,233919,Clover,15.6,Core,122.0,184.0,43.0,NaN,NaN,NaN,43.000000,NaN
2,2019,18,201918,2019-05-04,233919,Clover,15.6,Core,61.0,219.0,184.0,43.0,NaN,NaN,113.500000,99.702056
3,2019,19,201919,2019-05-11,233919,Clover,15.6,Core,33.0,133.0,219.0,184.0,43.0,NaN,148.666667,93.168306
4,2019,20,201920,2019-05-18,233919,Clover,15.6,Core,142.0,310.0,133.0,219.0,184.0,43.0,144.750000,76.473852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2023,14,202314,2023-04-08,107583,Clover,15.6,Core,16.0,504.0,414.0,275.0,307.0,364.0,340.000000,61.552146
28,2023,15,202315,2023-04-15,107583,Clover,15.6,Core,77.0,245.0,504.0,414.0,275.0,307.0,375.000000,104.540263
29,2023,16,202316,2023-04-22,107583,Clover,15.6,Core,136.0,443.0,245.0,504.0,414.0,275.0,359.500000,121.244931
30,2023,17,202317,2023-04-29,107583,Clover,15.6,Core,115.0,297.0,443.0,245.0,504.0,414.0,401.500000,110.870796


### Data Preparation

In [ ]:
# Data preparation
X_all = clean_data.drop(["inventory_units"], axis=1).dropna()
y_all = clean_data.dropna()["inventory_units"]

In [ ]:
X_all

,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units+1,inventory_units+2,inventory_units+3,inventory_units+4,Mean,std
4,2019,20,201920,2019-05-18,233919,Clover,15.6,Core,142.0,133.0,219.0,184.0,43.0,144.75,76.473852
5,2019,21,201921,2019-05-25,233919,Clover,15.6,Core,24.0,310.0,133.0,219.0,184.0,211.50,74.558702
6,2019,22,201922,2019-06-01,233919,Clover,15.6,Core,117.0,426.0,310.0,133.0,219.0,272.00,125.552114
7,2019,23,201923,2019-06-08,233919,Clover,15.6,Core,79.0,311.0,426.0,310.0,133.0,295.00,120.949025
8,2019,24,201924,2019-06-15,233919,Clover,15.6,Core,88.0,264.0,311.0,426.0,310.0,327.75,69.071823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2023,14,202314,2023-04-08,107583,Clover,15.6,Core,16.0,414.0,275.0,307.0,364.0,340.00,61.552146
28,2023,15,202315,2023-04-15,107583,Clover,15.6,Core,77.0,504.0,414.0,275.0,307.0,375.00,104.540263
29,2023,16,202316,2023-04-22,107583,Clover,15.6,Core,136.0,245.0,504.0,414.0,275.0,359.50,121.244931
30,2023,17,202317,2023-04-29,107583,Clover,15.6,Core,115.0,443.0,245.0,504.0,414.0,401.50,110.870796


In [ ]:
y_all

4     310.0
5     426.0
6     311.0
7     264.0
8     258.0
      ...  
27    504.0
28    245.0
29    443.0
30    297.0
31    242.0
Name: inventory_units, Length: 7653, dtype: float64

In [ ]:
def prepare_df(df):
    df = df[['product_number', 'prod_category','display_size', 'segment', 'sales_units', "inventory_units+1", "inventory_units+2", "inventory_units+3", "inventory_units+4", "Mean", "std"]]
    df = pd.get_dummies(df, columns=['product_number', 'prod_category','display_size', 'segment'])
    return df

In [ ]:
X_train = prepare_df(X_all)
y_train = y_all

### Model Training

In [ ]:

# Set random seed for as reproducible results as possible
tf.random.set_seed(42)

# Construct model
model_1 = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(1)      
], name="model_1_dense") # give the model a name so we can save it

# Compile model
model_1.compile(
                optimizer=tf.keras.optimizers.Adam(lr=0.001),
                loss='mse',
                metrics=[tf.keras.metrics.RootMeanSquaredError()]) 

model_1.fit(x=X_train, # train windows of 4 timesteps of maize prices
            y=y_train, # horizon value of 1
            epochs=100,
            verbose=1,
            batch_size=32,) # create ModelCheckpoint callback to save best model

In [ ]:
model_1.evaluate(X_train, y_train)

240/240 [==============================] - 1s 3ms/step - loss: 4192.0083 - root_mean_squared_error: 64.7457


[4192.00830078125, 64.74571990966797]

### Inference
we are going to predict the inventory units in the next 6 months for 100 products

In [ ]:
# we need to prepare the test data 
# and fill empty values before we can submit it
test_df = pd.read_csv('test.csv')
test_df[['year_week', 'product_number']] = test_df.id.str.split('-', expand = True)
test_df['product_number'] = test_df.product_number
test_df

,id,year_week,product_number
0,202319-105609,202319,105609
1,202319-107583,202319,107583
2,202319-10857,202319,10857
3,202319-109557,202319,109557
4,202319-112518,202319,112518
...,...,...,...
1295,202331-92778,202331,92778
1296,202331-93765,202331,93765
1297,202331-94752,202331,94752
1298,202331-9870,202331,9870


In [ ]:
# each product has specific information, we are just creating a reference table
product_mapping = X_all[['product_number', 'prod_category','display_size', 'segment']].drop_duplicates()
product_mapping

,product_number,prod_category,display_size,segment
4,233919,Clover,15.6,Core
4,235893,Clover,17.3,Core
4,234906,Goku,15.6,Gaming
4,236880,Goku,15.6,Core
4,247737,Arale,13.3,Premium
...,...,...,...,...
4,222075,Doraemon,16.1,Gaming
4,196413,Clover,15.6,Core
4,193452,Bobobo,16.1,Gaming
4,204309,Doraemon,16.1,Gaming


In [ ]:
test_df_complete = test_df.merge(product_mapping, on='product_number', how = 'left')
test_df_complete

,id,year_week,product_number,prod_category,display_size,segment
0,202319-105609,202319,105609,Goku,15.6,Gaming
1,202319-107583,202319,107583,Clover,15.6,Core
2,202319-10857,202319,10857,Doraemon,15.6,Gaming
3,202319-109557,202319,109557,Goku,14.0,Core
4,202319-112518,202319,112518,Conan,15.6,Premium
...,...,...,...,...,...,...
1295,202331-92778,202331,92778,Goku,14.0,Core
1296,202331-93765,202331,93765,Arale,14.0,Premium
1297,202331-94752,202331,94752,Clover,14.0,Core
1298,202331-9870,202331,9870,Doraemon,15.6,Gaming


In [ ]:
# Inventory Prediction
results = {}
for product_id in test_df_complete.product_number.unique():
    feature_values = []
    future_forecast = []
    da = test_df_complete[test_df_complete['product_number']==product_id]
    last_window = list(clean_data[clean_data['product_number']==product_id].inventory_units.values[-WINDOW_SIZE:])
    last_sales = list(clean_data[clean_data['product_number']==product_id].sales_units.values[-WINDOW_SIZE:])
    for i in range(13):
        features = da[['product_number', 'prod_category','display_size', 'segment']].head(1)
        features["inventory_units+1"] = last_window[3]
        features["inventory_units+2"] = last_window[2]
        features["inventory_units+3"] = last_window[1]
        features["inventory_units+4"] = last_window[0]
        features['Mean'] = features[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].mean(axis=1)
        features['std'] = features[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].std(axis=1)
        mean = np.mean(last_sales)
        features['sales_units'] = [mean]
        last_sales = (last_sales + [mean])[1:]
        prepare_feature = prepare_df(features)
        # Align columns of test set with training set
        missing_cols = set(X_train.columns) - set(prepare_feature.columns)
        for col in missing_cols:
            prepare_feature[col] = 0

        prepare_feature = prepare_feature[X_train.columns]
        pred_value = model_1.predict(prepare_feature)
        # Create an empty list for future forcast
        last_window = (list(pred_value[0]) + last_window)[:-1]
        future_forecast.append(round(pred_value[0][0]))

        # Create a empty list for future feature
        feature_values.append(last_window)
    
    results[product_id] = future_forecast

In [ ]:
# Save the results
import json
with open("result_final.json", "w") as fp:
    json.dump(results,fp) 

In [ ]:
def fix_date_test(data):
  data[2]="2023-05-13"
  data[2]=pd.to_datetime(data[2])
  sample = data.sort_values(0,ascending=True).reset_index(drop=True)
  for i in range(len(sample)-1):
    sample[2][i+1] = sample[2][i] + timedelta(weeks = 1)
  return sample

def prepare_test_data(test):
  test = test.id.str.split("-", expand = True)
  df_tst = []
  for product_id in test[1].unique():
      da = test[test[1]==product_id]
      ab = fix_date_test(da)
      df_tst.append(ab)
  test_data = pd.concat(df_tst, axis=0)
  return test_data
def get_to_submission(frame_test):
  frame_test.insert(loc=0, column='id', value=frame_test['week_year'] +"-"+frame_test['product_number'])
  finalresult = frame_test.drop(['product_number','week_year','index','date'],axis=1).reset_index(drop=True)
  finalresult.columns=['id','inventory_units']
  finalresult.to_csv('baseline_submission.csv',index=False)

In [ ]:
test = pd.read_csv('/content/test.csv')
test = prepare_test_data(test)

In [ ]:
test = prepare_test_data(test)

sd = []
test.columns=['week_year','product_number','date']
for i in range(len(test.week_year.unique())):
  submission = test[test['week_year']==test.week_year.unique()[i]].reset_index()
  submission['prediction']=0
  for (row, data) in submission.iterrows():
      submission['prediction'][row] = results[data['product_number']][i]
  sd.append(submission)
test_alo = pd.concat(sd, axis=0)
get_to_submission(test_alo)